In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = [([[0.575;-0.175] [0.575;0.175] [0.925;-0.175] [0.925;0.175]], [[0,1,2,3]])]

1-element Array{Tuple{Array{Float64,2},Array{Array{Int64,1},1}},1}:
 ([0.575 0.575 0.925 0.925; -0.175 0.175 -0.175 0.175], [[0, 1, 2, 3]])

## Vecchia funzione

In [3]:
function checkStruct(lst)
	obj = lst[1]
	if isa(obj,Matrix)
		dim = size(obj,1)-1
	elseif (isa(obj,Tuple) || isa(obj,Array))
		dim = length(obj[1][:,1])

	elseif isa(obj,Lar.Struct)
		dim = length(obj.box[1])
	end
	return dim
end

checkStruct (generic function with 1 method)

In [4]:
@btime checkStruct(square)

  81.923 ns (1 allocation: 96 bytes)


2

In [5]:
@benchmark checkStruct(square)

BenchmarkTools.Trial: 
  memory estimate:  96 bytes
  allocs estimate:  1
  --------------
  minimum time:     80.063 ns (0.00% GC)
  median time:      82.437 ns (0.00% GC)
  mean time:        104.591 ns (5.64% GC)
  maximum time:     3.998 μs (95.46% GC)
  --------------
  samples:          10000
  evals/sample:     968

In [6]:
@code_typed checkStruct(square)

CodeInfo(
1 ── %1  = Base.arrayref(true, lst, 1)::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
└───       goto #3 if not false
2 ──       nothing::Nothing
3 ┄─       nothing::Nothing
│    %5  = Base.getfield(%1, 1, true)::Array{Float64,2}
│    %6  = Base.arraysize(%5, 1)::Int64
│          Base.arraysize(%5, 2)::Int64
│    %8  = Base.slt_int(%6, 0)::Bool
│    %9  = Base.ifelse(%8, 0, %6)::Int64
│    %10 = %new(Base.OneTo{Int64}, %9)::Base.OneTo{Int64}
│    %11 = %new(Base.Slice{Base.OneTo{Int64}}, %10)::Base.Slice{Base.OneTo{Int64}}
└───       goto #8 if not true
4 ── %13 = Core.tuple(%11, 1)::Tuple{Base.Slice{Base.OneTo{Int64}},Int64}
│          Base.arraysize(%5, 1)::Int64
│    %15 = Base.arraysize(%5, 2)::Int64
│    %16 = Base.slt_int(%15, 0)::Bool
│    %17 = Base.ifelse(%16, 0, %15)::Int64
│    %18 = Base.sle_int(1, 1)::Bool
│    %19 = Base.sle_int(1, %17)::Bool
│    %20 = Base.and_int(%18, %19)::Bool
│    %21 = Base.and_int(%20, true)::Bool
│    %22 = Base.and_int(true, %21)::Bo

In [7]:
@code_warntype checkStruct(square)

Variables
  #self#::Core.Compiler.Const(checkStruct, false)
  lst::Array{Tuple{Array{Float64,2},Array{Array{Int64,1},1}},1}
  obj::Tuple{Array{Float64,2},Array{Array{Int64,1},1}}
  dim::Int64
  @_5::Bool

Body::Int64
1 ─       Core.NewvarNode(:(dim))
│         (obj = Base.getindex(lst, 1))
│   %3  = (obj isa Main.Matrix)::Core.Compiler.Const(false, false)
└──       goto #3 if not %3
2 ─       Core.Compiler.Const(:(Main.size(obj, 1)), false)
│         Core.Compiler.Const(:(dim = %5 - 1), false)
└──       Core.Compiler.Const(:(goto %25), false)
3 ┄ %8  = (obj isa Main.Tuple)::Core.Compiler.Const(true, false)
│         %8
│         (@_5 = %8)
└──       goto #5
4 ─       Core.Compiler.Const(:(@_5 = obj isa Main.Array), false)
5 ┄       @_5::Core.Compiler.Const(true, false)
│   %14 = Base.getindex(obj, 1)::Array{Float64,2}
│   %15 = Base.getindex(%14, Main.:(:), 1)::Array{Float64,1}
│         (dim = Main.length(%15))
└──       goto #7
6 ─       Core.Compiler.Const(:(obj), false)
│         C

## Modifiche

In [5]:
function checkStruct(lst)
	obj = lst[1]
	if isa(obj,Matrix)
		dim = size(obj,1)-1
	elseif (isa(obj,Tuple) || isa(obj,Array))
		dim = length(@view (obj[1][:,1]))

	elseif isa(obj,Lar.Struct)
		dim = length(obj.box[1])
	end
	return dim
end

checkStruct (generic function with 1 method)

In [6]:
@btime checkStruct(square)

  34.240 ns (0 allocations: 0 bytes)


2